## Imports

In [50]:
import os
import pickle

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, KFold
from sklearn.model_selection import StratifiedKFold, cross_val_score


# Data

In [51]:
data = pd.read_csv('new_Exam_Score_Prediction.csv')

y = data['exam_score']
X = data.drop('exam_score', axis=1)

data

,age,gender,study_hours,class_attendance,internet_access,sleep_hours,sleep_quality,facility_rating,exam_difficulty,exam_score,diploma,group_study,self_study,online_videos,coaching,mixed
0,17,1,2.78,92.9,1,7.4,0,0,2,58.9,1,0,0,0,1,0
1,22,1,7.88,76.8,1,8.5,0,2,1,90.3,0,0,0,0,1,0
2,20,0,0.89,71.6,1,9.8,0,0,1,43.7,1,0,0,0,1,0
3,23,1,3.48,65.4,1,4.2,2,0,1,58.2,0,0,0,0,0,1
4,17,0,1.35,69.0,1,7.4,1,2,2,53.7,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13269,17,1,5.74,81.4,1,7.5,2,2,2,95.0,0,0,0,1,0,0
13270,19,0,3.28,73.3,1,9.5,2,2,0,93.9,0,1,0,0,0,0
13271,18,1,3.71,41.6,0,5.9,1,1,1,60.9,0,0,0,0,1,0
13272,19,1,4.60,76.3,0,6.1,2,1,1,79.0,0,0,1,0,0,0


# Models

In [52]:
PATH = 'pipes'
pipe_names = os.listdir(PATH)[:-1] #Gitignore file to not pushing the models

pipes = {}

for name in pipe_names:
    with open(os.path.join(PATH, name), 'rb') as f:
        pipes[name.strip('.pkl')] = pickle.load(f)

pipes

{'XSB': Pipeline(steps=[('scaler', StandardScaler()),
                 ('features_selector',
                  SelectKBest(k=15,
                              score_func=<function f_regression at 0x7f26ce1c0680>)),
                 ('regressor',
                  XGBRegressor(base_score=None, booster=None, callbacks=None,
                               colsample_bylevel=None, colsample_bynode=None,
                               colsample_bytree=0.8207886710429086, device=None,
                               early_stopping_rounds=None,
                               enable_categorical=False, e...
                               gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None,
                               learning_rate=0.0059354140207009305, max_bin=None,
                               max_cat_threshold=None, max_cat_to_onehot=None,
                               max_delta_step=None, max_dept

# Friedman test

## Results for testing

In [53]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

#TODO: Get models score
results = {i: [] for i in range(len(pipes))}

for i in range(10):
    # 1. Create a unique split for this "block"
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=i # Use i to ensure different splits
    )

    for idx, model in enumerate(pipes.values()):
        # print(type(model))
        # 2. Fit the loaded instance directly
        model.fit(X_train, y_train)

        # 3. Predict and score
        preds = model.predict(X_test)
        score = r2_score(y_test, preds)

        # 4. Store the score
        results[idx].append(score)

        # 4. Store the score
#TODO Friedman TEST



results

<class 'sklearn.pipeline.Pipeline'>
<class 'sklearn.pipeline.Pipeline'>
<class 'sklearn.pipeline.Pipeline'>
<class 'sklearn.pipeline.Pipeline'>
<class 'sklearn.pipeline.Pipeline'>
<class 'sklearn.pipeline.Pipeline'>
<class 'sklearn.pipeline.Pipeline'>
<class 'sklearn.pipeline.Pipeline'>
<class 'sklearn.pipeline.Pipeline'>
<class 'sklearn.pipeline.Pipeline'>
<class 'sklearn.pipeline.Pipeline'>
<class 'sklearn.pipeline.Pipeline'>
<class 'sklearn.pipeline.Pipeline'>
<class 'sklearn.pipeline.Pipeline'>
<class 'sklearn.pipeline.Pipeline'>
<class 'sklearn.pipeline.Pipeline'>
<class 'sklearn.pipeline.Pipeline'>
<class 'sklearn.pipeline.Pipeline'>
<class 'sklearn.pipeline.Pipeline'>
<class 'sklearn.pipeline.Pipeline'>
<class 'sklearn.pipeline.Pipeline'>
<class 'sklearn.pipeline.Pipeline'>
<class 'sklearn.pipeline.Pipeline'>
<class 'sklearn.pipeline.Pipeline'>
<class 'sklearn.pipeline.Pipeline'>
<class 'sklearn.pipeline.Pipeline'>
<class 'sklearn.pipeline.Pipeline'>
<class 'sklearn.pipeline.Pip

{0: [0.7161475412780569,
  0.7281916784685578,
  0.7148020828118922,
  0.7063865715265675,
  0.723712650616388,
  0.7139676599454685,
  0.7214548392543558,
  0.7258825820448086,
  0.7157940878790179,
  0.7120925141019121],
 1: [0.63183464456141,
  0.651889530405483,
  0.6336015431211766,
  0.6259123820150619,
  0.6363020299432223,
  0.633651991822141,
  0.6390306387439733,
  0.6423649933818019,
  0.6325266179054474,
  0.6340875586470495],
 2: [0.7164899506125981,
  0.7277855301222731,
  0.7206985287263867,
  0.7108970879724763,
  0.7210488985783279,
  0.7128666911979582,
  0.7183485133529559,
  0.7257418405936358,
  0.7130518487253509,
  0.7133541709286466],
 3: [0.6951398345188092,
  0.7113623089151867,
  0.6992835569952924,
  0.6873950988439873,
  0.7034491121460698,
  0.6965740620163388,
  0.7043935058794966,
  0.7051606794855882,
  0.6984550368152358,
  0.6964869608159583],
 4: [0.7248783185904926,
  0.7377017783294133,
  0.7274060396716573,
  0.7182134796810176,
  0.73233834413985